# Deep Learning with PyTorch : Build a Generative Adversarial Network

In [ ]:
import torch  #Torch is imported for tensors operations
torch.manual_seed(42)
import numpy as np #Numpy for numerical computationsacc
import matplotlib.pyplot as plt #Matplotlib for plotting

from tqdm.notebook import tqdma # For showing Progress Bar

# Configurations

In [ ]:
device = 'cuda' # Transfer images to gpu device
batch_size = 128 # Size defined to be used in training loop
noise_dim = 64  # The noise which we will give to generator

epochs= 20 # How many times training loop is going to run

# Hyper Parameters for Adam Optimizer
beta_1 = 0.5
beta_2 = 0.99
lr = 0.0002



# Load MNIST Dataset

In [ ]:
from torchvision import datasets,transforms as T

In [ ]:
import torchvision.transforms as T

train_augs = T.Compose([
    T.RandomRotation((-20, 20)),  # Random rotation between -20 and +20 degrees, This can help introduce variety and robustness to the training data.
    T.ToTensor()  # Convert images to PyTorch tensors
])



In [ ]:
trainset=datasets.MNIST('MNIST/',download= True , train= True,transform=train_augs)

In [ ]:
image,label= trainset[800] # Loading image and it's coresponding label.

plt.imshow(image.squeeze(),cmap='gray') # Plot a single Image and it's squezed to remove any single dimensional array

# Load Dataset Into Batches

In [ ]:
from torch.utils.data import DataLoader # It creates batches for training
from torchvision.utils import make_grid # create a grid of images for visualization

In [ ]:
trainloader= DataLoader(trainset,batch_size=batch_size,shuffle=True) # This line creates a data loader that loads training data in mini-batches. The data is shuffled

In [ ]:
print("The total no of batches are: ",trainloader)

In [ ]:
dataiter = iter(trainloader) # function is used to create an iterator from the data loader. An iterator is an object that allows you to iterate through the data loader and retrieve batches of data.
images, _ = next(dataiter) # unction is used to get the next batch of data from the iterator
print(images.shape)

#(batch_size, channels, height, width)


In [ ]:
# 'show_tensor_images' : function is used to plot some of images from the batch
# The function converts the tensor to a NumPy array and creates a grid of images using make_grid, then displays it using matplotlib.

def show_tensor_images(tensor_img, num_images = 16, size=(1, 28, 28)):
    unflat_img = tensor_img.detach().cpu()
    img_grid = make_grid(unflat_img[:num_images], nrow=4)
    plt.imshow(img_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [ ]:
show_tensor_images(images,num_images = 16)

# Create Discriminator Network

In [ ]:
!pip install torchsummary

In [ ]:
from torch import nn # importing neural networks
from torchsummary import summary

In [ ]:
from torch.nn.modules.activation import LeakyReLU
from torch.nn.modules.batchnorm import BatchNorm2d

# Returns a Sequential Block
def get_disc_block(in_channel,out_channel,kernel_size,stride):
  return nn.Sequential(
      nn.Conv2d(in_channel,out_channel,kernel_size,stride),
      nn.BatchNorm2d(out_channel),
      nn.LeakyReLU(0.2)
  )

In [ ]:
class Discriminator(nn.Module):  # This class Discriminator is Inherited from NN MODULE
# NN MODULE IS THE BASE CLASS OF ALL
  def __init__(self):
    super(Discriminator,self).__init__()  #  This line ensures that the constructor of the parent class (nn.Module) is called.

    self.block1=get_disc_block(1,16,(3,3),2)
    self.block2=get_disc_block(16,32,(5,5),2)
    self.block3=get_disc_block(32,64,(5,5),2)

    self.flatten=nn.Flatten() # flatten the output of the convolutional blocks before passing it to the linear layer.
    self.linear=nn.Linear(in_features = 64, out_features = 1)

  def forward(self,images):

    x1=self.block1(images)
    x2=self.block2(x1)
    x3=self.block3(x2)

    x4=self.flatten(x3)
    x5=self.linear(x4)

    return x5



In [ ]:
# NETWORK SUMMARY

D=Discriminator()
D.to(device)

summary(D,input_size=(1,28,28))

# Create Generator Network

In [ ]:
def get_gen_block(in_channels,out_channels,kernel_size,stride,final_block=False):
  if final_block == True:
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels,out_channels,kernel_size,stride),
        nn.Tanh()
    )

  return nn.Sequential(
      nn.ConvTranspose2d(in_channels,out_channels,kernel_size,stride),
      nn.BatchNorm2d(out_channels),
      nn.ReLU()
  )

In [ ]:
class Generator(nn.Module):


  def __init__(self,noise_dim):
    super(Generator,self).__init__()   # This line ensures that the constructor of the parent class (nn.Module) is called.

    self.noise_dim=noise_dim #Dimensionality of the input noise vector. This noise will be used to generate images.

    self.block1=get_gen_block(noise_dim,256 ,(3,3),2)
    self.block2=get_gen_block(256,128, (4,4),1)
    self.block3=get_gen_block(128,64, (3,3),2)

    self.block4=get_gen_block(64,1, (4,4),2,final_block=True)

  def forward(self,r_noise_vec):

    x=r_noise_vec.view(-1,self.noise_dim,1,1) #The input noise is reshaped to a 4D tensor with dimensions (batch_size, noise_dim, 1, 1).
    # view operation reshapes the input noise tensor from (batch_size, noise_dim) to (batch_size, noise_dim, 1, 1)

    x1=self.block1(x)
    x2=self.block2(x1)
    x3=self.block3(x2)
    x4=self.block4(x3)

    return x4

In [ ]:
G=Generator(noise_dim)
G.to(device)

summary(G,input_size=(1,noise_dim))

In [ ]:
# Replace Random initialized weights to Normal weights

def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight, 0.0, 0.02)
        nn.init.constant_(m.bias, 0)

In [ ]:
D=D.apply(weights_init)
G=G.apply(weights_init)

# Create Loss Function and Load Optimizer

In [ ]:
def real_loss(disc_pred):
  criterion=nn.BCEWithLogitsLoss()
  ground_truth=torch.ones_like(disc_pred)
  loss=criterion(disc_pred,ground_truth)
  return loss

def fake_loss(disc_pred):
  criterion=nn.BCEWithLogitsLoss()
  ground_truth=torch.zeros_like(disc_pred)
  loss=criterion(disc_pred,ground_truth)
  return loss


In [ ]:
# optimizer for training the discriminator
#D.parameters(): This passes the parameters of the discriminator model (D) to the optimizer. It allows the optimizer to update the weights and biases of the discriminator during training.
#hese are the beta parameters used in the Adam optimizer. beta_1 and beta_2 control the exponential moving averages of gradients and squared gradients, respectively.
D_opt=torch.optim.Adam(D.parameters(),lr=lr,betas=(beta_1,beta_2))
G_opt=torch.optim.Adam(G.parameters(),lr=lr,betas=(beta_1,beta_2))

# Training Loop

In [ ]:
# This big loop goes through a set number of rounds called "epochs".
#For each epoch, there are two counters: one for adding up the losses of the discriminator (a part of a computer program), and another for adding up the losses of the generator (another part of the program).
#Inside each epoch, there's a smaller loop that looks at the training data. This data is taken in chunks, and each chunk has real pictures and their labels (like tags) that are not really used here.
#The real pictures are moved to where they need to be processed, which can be either the main computer part (CPU) or a special powerful part (GPU).
#Also, random noise is made using a math function. This noise is like random data that will be used to create fake pictures by another part of the program.


for i in range(epochs):

  total_d_loss = 0.0
  total_g_loss = 0.0

  for real_img, _ in trainloader:

    real_img=real_img.to(device)
    noise = torch.randn(batch_size,noise_dim,device=device)

    # FIND LOSS AND UPDATE WEIGHT FOR D

    D_opt.zero_grad() # Gradients are reset

    fake_img=G(noise) #  generator creates fake images

    D_pred=D(fake_img) # discriminator makes predictions (D_pred) on the fake images
    D_fake_loss=fake_loss(D_pred) # fake loss is calculated using the fake_loss function.

    D_pred=D(real_img) # he discriminator makes predictions on the real images
    D_real_loss=real_loss(D_pred) #  real loss is calculated using the real_loss function.

    D_loss= (D_fake_loss + D_real_loss) /2 # The final discriminator loss (D_loss) is calculated as the average of the fake and real losses.


    total_d_loss += D_loss.item()

    D_loss.backward()# The discriminator loss is backpropagated
    D_opt.step()

    # FIND LOSS AND UPDATE WEIGHT FOR G

    G_opt.zero_grad()

    noise= torch.randn(batch_size,noise_dim,device=device)

    fake_img=G(noise)
    D_pred= D(fake_img)
    G_loss = real_loss (D_pred)

    total_g_loss += G_loss.item()

    G_loss.backward()
    G_opt.step()

avg_d_loss= total_d_loss / len(trainloader)
avg_g_loss= total_g_loss / len(trainloader)

print("Epoch : {} | Dloss : {} | Gloss : {}".format(i+1,avg_d_loss,avg_g_loss))

show_tensor_images(fake_img)